# Assigning children to sandwiches

A mother needs to assign each of her 5 children to 5 different sandwiches. Each child has given a preference for each sandwich by rating it from 0 to 10 (0 indicates strong dislike and 10 indicates strong preference). The mother wishes to assign sandwiches in such a way as to maximize the overall satisfaction. The preferences of the children are as follows:

|Sandwich| Akhil | Brianna | Carlos | Dinah | Edward |
|-------:|------:|--------:|-------:|------:|-------:|
|  PB&J   | 2     |   9     |  1     | 9     | 6      |  
| Turkey | 9     |   0     |  5     | 5     | 8      |  
|  Ham   | 7     |   0     |  10    | 2     | 6      |
| Veggie | 0     |   9     |  8     | 1     | 7      |
| Tuna   | 4     |   1     |  4     | 0     | 4      |


In [1]:
using JuMP, Clp, NamedArrays

# create the index sets (sandwich types, children)
sandwiches = [ :PBJ, :Turkey, :Ham, :Veggie, :Tuna ]
kids = [ :Akhil, :Brianna, :Carlos, :Dinah, :Edward ]

# create the input matrix for a NamedArray that is indexed by sandwiches (rows)
# and children (columns) with elements showing the preferences
prefs = [ 2 9 1 9 6
        9 0 5 5 8
        7 0 10 2 6
        0 9 8 1 7
        4 1 4 0 4]

# create a NamedArray showing each child's sandwich preferences
preferences = NamedArray( prefs, (sandwiches,kids), ("sandwich","kid"))

m = Model(Clp.Optimizer)

# variable for assigning each sandwich to a child
# NOTE: This is just a normal variable! We don't need to specify "binary"
# in this case. We'll see why soon.
@variable(m, x[sandwiches,kids] >= 0)

# each kid gets exactly 1 sandwich
@constraint(m, a[j in kids], sum(x[i,j] for i in sandwiches) == 1 )

# each sandwich gets assigned to one kid
@constraint(m, b[i in sandwiches], sum(x[i,j] for j in kids) == 1 )

# maximize overall preference
@objective(m, Max, sum( x[i,j]*preferences[i,j] for i in sandwiches, j in kids ) )

optimize!(m)

# we can print the solution to a NamedArray for ease of understanding
assignment = NamedArray( [ (value(x[i,j])) for i in sandwiches, j in kids ], (sandwiches, kids), ("sandwich","kid"))



Coin0506I Presolve 10 (0) rows, 25 (0) columns and 50 (0) elements
Clp0006I 0  Obj 0 Primal inf 9.999999 (10) Dual inf 117 (21)
Clp0006I 7  Obj 41
Clp0000I Optimal - objective value 41
Clp0032I Optimal objective 41 - 7 iterations time 0.012


5×5 Named Matrix{Float64}
sandwich ╲ kid │   :Akhil  :Brianna   :Carlos    :Dinah   :Edward
───────────────┼─────────────────────────────────────────────────
:PBJ           │      0.0       0.0       0.0       1.0       0.0
:Turkey        │      1.0       0.0       0.0       0.0       0.0
:Ham           │      0.0       0.0       1.0       0.0       0.0
:Veggie        │      0.0       1.0       0.0       0.0       0.0
:Tuna          │      0.0       0.0       0.0       0.0       1.0

Notice that even though we didn't tell the variable it had to be binary, it is in the solution! Hmm....

Interesting side-note: Being near the middle on all the rankings caused Edward to end up with his least-favorite sandwich! Let's see what happens if he changes his ratings to be more extreme:

|Sandwich| Akhil | Brianna | Carlos | Dinah | Edward |
|-------:|------:|--------:|-------:|------:|-------:|
|  PB&J   | 2     |   9     |  1     | 9     | 6      |  
| Turkey | 9     |   0     |  5     | 5     | 9      |  
|  Ham   | 7     |   0     |  10    | 2     | 6      |
| Veggie | 0     |   9     |  8     | 1     | 7      |
| Tuna   | 4     |   1     |  4     | 0     | 3      |

In [2]:
prefs = [ 2 9 1 9 6
        9 0 5 5 9
        7 0 10 2 6
        0 9 8 1 7
        4 1 4 0 3]

preferences = NamedArray( prefs, (sandwiches,kids), ("sandwich","kid"))

m = Model(Clp.Optimizer)

# NOTE: This is just a normal variable! We don't need to specify "binary"
# in this case. We'll see why soon.
@variable(m, x[sandwiches,kids] >= 0)

# each kid gets exactly 1 sandwich
@constraint(m, a[j in kids], sum(x[i,j] for i in sandwiches) == 1 )

# each sandwich gets assigned to one kid
@constraint(m, b[i in sandwiches], sum(x[i,j] for j in kids) == 1 )

@objective(m, Max, sum( x[i,j]*preferences[i,j] for i in sandwiches, j in kids ) )

optimize!(m)

assignment = NamedArray( [ (value(x[i,j])) for i in sandwiches, j in kids ], (sandwiches, kids), ("sandwich","kid"))

Coin0506I Presolve 10 (0) rows, 25 (0) columns and 50 (0) elements
Clp0006I 0  Obj 0 Primal inf 9.999999 (10) Dual inf 117 (21)
Clp0006I 7  Obj 41
Clp0000I Optimal - objective value 41
Clp0032I Optimal objective 41 - 7 iterations time 0.002


5×5 Named Matrix{Float64}
sandwich ╲ kid │   :Akhil  :Brianna   :Carlos    :Dinah   :Edward
───────────────┼─────────────────────────────────────────────────
:PBJ           │      0.0       0.0       0.0       1.0       0.0
:Turkey        │      0.0       0.0       0.0       0.0       1.0
:Ham           │      0.0       0.0       1.0       0.0       0.0
:Veggie        │      0.0       1.0       0.0       0.0       0.0
:Tuna          │      1.0       0.0       0.0       0.0       0.0